In [1]:
#!pip install spacy
#!python -m spacy download en_core_web_lg

We are using the pretrained models of spacy large models.
Also we are using word2vec for sentiment analysis of imdb movies reviews of labelled dataset.

In [2]:
import en_core_web_lg

In [3]:
nlp=en_core_web_lg.load()

Now we will undestand how the similarity between tokens of same sentence work in spacy.

In [4]:
#below is the examples just for understanding
x= 'dog cat train lion erdga'
doc=nlp(x)

for token1 in doc:
  for token2 in doc:
    print(token1.text, token2.text,token1.similarity(token2))



dog dog 1.0
dog cat 0.80168545
dog train 0.35094148
dog lion 0.47424486
dog erdga 0.0
cat dog 0.80168545
cat cat 1.0
cat train 0.26588535
cat lion 0.5265438
cat erdga 0.0
train dog 0.35094148
train cat 0.26588535
train train 1.0
train lion 0.24996218
train erdga 0.0
lion dog 0.47424486
lion cat 0.5265438
lion train 0.24996218
lion lion 1.0
lion erdga 0.0
erdga dog 0.0
erdga cat 0.0
erdga train 0.0
erdga lion 0.0
erdga erdga 1.0


/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty v

We are getting above warnings because we are using erdga in our sentence and since that is not a valid token inside the vocabulary of the large spacy model therefore it is mentioning in the error that the similarity is based on empty vector for OOV(out of vocabulary) token.

Explanation:-

dog cat is showing 80 percent similarity in the sense that both are tamed animals and most of the times they might be used in same sentence for comparasion etc.

cat and lion are showing 52 percent similarity as both of them belong to same family etc.

dog train cat train have relatively low similarities as compared to others because they have no relation in between them

**Model buliding for the problem statement**

In [5]:
import numpy as np
import pandas as pd


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report,confusion_matrix

In [120]:
df=pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/moviereviews.tsv',sep='\t')

In [121]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [63]:
import spacy
nlp=spacy.load('en_core_web_lg',disable=['parser', 'ner'])

In [64]:
STOPWORDS=nlp.Defaults.stop_words

In [65]:
import re
import string

In [51]:
list(STOPWORDS)

['their',
 'him',
 'very',
 'neither',
 'indeed',
 'fifteen',
 '‘ll',
 'thus',
 'across',
 'might',
 'please',
 'moreover',
 'it',
 'which',
 'out',
 'next',
 'been',
 'the',
 'not',
 'your',
 'yourselves',
 'various',
 'had',
 'that',
 'some',
 "'ve",
 '‘re',
 'they',
 'see',
 'while',
 'who',
 'hereby',
 'among',
 'mostly',
 'often',
 'become',
 'them',
 'where',
 'before',
 'my',
 'used',
 'whatever',
 'side',
 '’d',
 'ourselves',
 'whereby',
 'although',
 'us',
 'therefore',
 'whether',
 'seemed',
 "'s",
 'eleven',
 'through',
 'ten',
 'would',
 'ours',
 'became',
 'may',
 'made',
 'without',
 'most',
 'seeming',
 '’ve',
 'along',
 'never',
 'seem',
 'besides',
 'was',
 'still',
 'somewhere',
 'bottom',
 'sometimes',
 'herein',
 'call',
 "'d",
 'always',
 'anywhere',
 'everywhere',
 'this',
 'amount',
 'why',
 'could',
 'did',
 'themselves',
 'really',
 'and',
 'again',
 'otherwise',
 'also',
 'throughout',
 'else',
 'being',
 'onto',
 'forty',
 'regarding',
 'for',
 'third',
 'her

In [127]:
#preprocessing steps
def clean_data_set(x):
  #removing specials charaters from the sentence
  x=re.sub(r'[^\w ]+', "", str(x))
  
  doc=nlp(x)
  #lemmatization
  x=[token.lemma_ for token in doc ]
  #conversion to lower case
  x=[str(token).lower() for token in x]
  #removal of stopwords
  x=[token for token in x if token not in STOPWORDS]
  #removal of numbers  
  x=[token for token in x if not re.search('[0-9].*',token)]
  x=[token for token in x if token!='-pron-']
  x=' '.join(x)
  return x

  
  


In [128]:
df['review']=df['review'].apply(clean_data_set)

In [129]:
df.head()

,label,review
0,neg,film like mouse hunt theatre law diabolica...
1,neg,talented actress bless demonstrate wide acting...
2,pos,extraordinary year australian film shine ...
3,pos,accord hollywood movie decade life small tow...
4,neg,press screening prime candidate bad year list ...


Word2Vec conversion below now.

What is a vector in terms of NLP?

Computer does not understand/process string characters only understand numeric ones. Therefore it is necessary to convert these into numeric representation.

The number of dimensions represent no of ways a particular word/vector can be represented. Since word2vec is a NN and the vectors are calulated by mathematical operations one cannot be sure of what each dimension represents 

In [130]:
def get_vec(x):
  doc=nlp(x)
  vec=doc.vector
  return vec

In [131]:
df['vec']=df['review'].apply(lambda x : get_vec(x))

In [153]:
df.head()

,label,review,vec
0,neg,film like mouse hunt theatre law diabolica...,"[-0.05718131, -0.0139228795, -0.0357752, -0.02..."
1,neg,talented actress bless demonstrate wide acting...,"[-0.051694006, 0.08378091, -0.056704335, -0.00..."
2,pos,extraordinary year australian film shine ...,"[-0.013633819, 0.062129304, -0.019596862, -0.0..."
3,pos,accord hollywood movie decade life small tow...,"[0.035942625, 0.032176398, 0.01065731, 0.02000..."
4,neg,press screening prime candidate bad year list ...,"[-0.024637252, 0.03194708, -0.063816905, -0.00..."


In [159]:
#converting to a numpy array

X=df['vec'].to_numpy()
X=X.reshape(-1,1)


In [160]:
X.shape

(2000, 1)

In [161]:
X=np.concatenate(X,axis=0)

In [162]:
X

array([array([-5.71813099e-02, -1.39228795e-02, -3.57751995e-02, -2.61799786e-02,
       -2.16462594e-02,  1.63589884e-02,  4.30012308e-02, -1.36611179e-01,
        3.87606323e-02,  1.42046583e+00, -7.90883601e-02, -4.67250720e-02,
       -2.06579566e-02,  2.81155836e-02, -4.01223935e-02,  1.84204318e-02,
       -4.83732810e-03,  5.14249980e-01, -1.37454532e-02, -3.01842997e-03,
        1.07199140e-02,  5.32269990e-03,  1.63153298e-02, -3.19665708e-02,
        6.43265434e-03,  9.52190347e-03, -4.20761630e-02, -5.36383800e-02,
        2.52457783e-02, -3.98481935e-02, -5.93247265e-02, -4.10275385e-02,
       -1.10948205e-01,  1.15025833e-01,  5.20179421e-02, -1.11289080e-02,
        4.15156074e-02,  6.46287575e-02, -7.65119866e-02,  4.01903987e-02,
        1.95610616e-02,  3.04015111e-02, -5.36678918e-02, -6.33278191e-02,
        9.73960608e-02, -6.64524967e-03,  1.08670043e-02,  7.44603481e-03,
        1.27520636e-02,  1.82524435e-02,  5.63356094e-02,  3.50789055e-02,
        2.42845844

In [163]:
X=np.concatenate(X,axis=0)

In [165]:
X.shape

(600000,)

In [167]:
X=X.reshape(-1,300)

In [168]:
X.shape

(2000, 300)

In [170]:
y=df['label']

In [171]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)  

In [172]:
X_test.shape

(400, 300)

Data prepared. Now going ahead with Model training

In [173]:
clf=LogisticRegression(solver='liblinear')

In [175]:
clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [176]:
y_pred=clf.predict(X_test)

In [177]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         neg       0.80      0.85      0.83       202
         pos       0.84      0.79      0.81       198

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400



In [178]:
import pickle


In [179]:
pickle.dump(clf,open('model_sentiment.pkl','wb'))